In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/amos.jerome1994/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="amos.jerome1994"
os.environ["MLFLOW_TRACKING_PASSWORD"]="5bf17de847045db9bf7d1ffe08b0779a6c8dceca"

In [2]:
import os
%pwd


'/Users/jeromeamos/Desktop/icare/MLops/datascienceproject/research'

In [3]:
os.chdir("../")
%pwd

'/Users/jeromeamos/Desktop/icare/MLops/datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/amos.jerome1994/datascienceproject.mlflow"
        )
        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-08 11:14:27,485:  INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-08 11:14:27,486:  INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-08 11:14:27,487:  INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-08 11:14:27,487:  INFO: common: created directory at: artifacts]
[2025-01-08 11:14:27,488:  INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-08 11:14:27,800:  INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/01/08 11:14:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/01/08 11:14:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


🏃 View run bold-stag-782 at: https://dagshub.com/amos.jerome1994/datascienceproject.mlflow/#/experiments/0/runs/07ca4b9bb3b84098822c2e6fde7375a2
🧪 View experiment at: https://dagshub.com/amos.jerome1994/datascienceproject.mlflow/#/experiments/0
